In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import tensorflow.compat.v1 as tf

In [3]:
from tensorflow import keras

In [4]:
from tensorflow.keras import backend as K

In [5]:
model_1 = keras.models.Sequential([
    keras.layers.Conv2D(filters=2, kernel_size=1, input_shape=(1,1,4)),
    keras.layers.Conv2D(filters=4, kernel_size=1)
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
model_1.inputs

[<tf.Tensor 'conv2d_input:0' shape=(?, 1, 1, 4) dtype=float32>]

In [7]:
model_1.outputs

[<tf.Tensor 'conv2d_1/BiasAdd:0' shape=(?, 1, 1, 4) dtype=float32>]

In [8]:
model_2 = keras.models.Sequential([
    keras.layers.Conv2D(filters=3, kernel_size=1, input_shape=(1,1,6)),
    keras.layers.Conv2D(filters=6, kernel_size=1)
])

In [9]:
model_2.inputs

[<tf.Tensor 'conv2d_2_input:0' shape=(?, 1, 1, 6) dtype=float32>]

In [10]:
model_2.outputs

[<tf.Tensor 'conv2d_3/BiasAdd:0' shape=(?, 1, 1, 6) dtype=float32>]

In [11]:
model_1_input_0 = model_1.inputs[0]
model_1_output_0 = model_1.outputs[0]
model_2_input_0 = model_2.inputs[0]
model_2_output_0 = model_2.outputs[0]

In [12]:
model_1_rms = K.sqrt(K.mean(K.square(model_1_output_0 - model_1_input_0), keepdims=True))

In [13]:
model_2_rms = K.sqrt(K.mean(K.square(model_2_output_0 - model_2_input_0), keepdims=True))

In [14]:
concat_rms = keras.layers.concatenate([model_1_rms, model_2_rms], axis=-1)

In [17]:
output_model = keras.models.Sequential([
    keras.layers.Conv2D(filters=1, kernel_size=1),
    keras.layers.Conv2D(filters=2, kernel_size=1)
])

In [18]:
output_model_ouput = output_model(concat_rms)

In [19]:
output_model_ouput

<tf.Tensor 'sequential_3/conv2d_7/BiasAdd:0' shape=(1, 1, 1, 2) dtype=float32>

In [20]:
output_model_rms = K.sqrt(K.mean(K.square(concat_rms - output_model_ouput)))

In [22]:
total_model = keras.Model([model_1_input_0, model_2_input_0], output_model_rms)

In [25]:
total_model.save("total.h5", save_format='h5')

ValueError: Unable to create group (name already exists)

In [31]:
total_model.layers[7].name

'tf_op_layer_sub_1'

In [36]:
total_model.outputs

[<tf.Tensor 'Sqrt_2:0' shape=() dtype=float32>]

In [32]:
tf_session = keras.backend.get_session()

# write out tensorflow checkpoint & meta graph
saver = tf.train.Saver()
save_path = saver.save(tf_session,"total/total.ckpt")

In [37]:
!CUDA_VISIBLE_DEVICES=-1 python freeze_graph.py \
    --input_meta_graph total/total.ckpt.meta \
    --input_checkpoint total/total.ckpt \
    --output_graph total/total.ckpt.pb \
    --output_node_names=Sqrt_2 \
    --input_binary=true

Loaded meta graph file 'total/total.ckpt.meta
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W1204 02:39:58.207679 139631774762816 deprecation.py:323] From freeze_graph.py:129: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2020-12-04 02:39:58.209058: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-04 02:39:58.229786: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-04 02:39:58.229822: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: giangblackk-hp-pavilion-gaming-laptop-15-dk0xxx
2020-12-04 02:39:58.229828: I tensorflow/stream_executor/cuda/cuda_diagnostics.c